In [1]:
import ee

# Initialize the Earth Engine module.
ee.Initialize()
import folium


def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [81]:
dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').first().select('avg_rad')


SEAsia_extent = ee.Image("users/ruarai/SEAsia_extent")

#dataset = dataset.unmask(dataset.mask().add(1))


dataset = dataset.reduceResolution(reducer = ee.Reducer.mean(),
                                             maxPixels = 65535)

dataset = dataset.reproject(crs = SEAsia_extent.projection())

dataset = dataset.clip(SEAsia_extent.geometry())

dataset = dataset.updateMask(SEAsia_extent.eq(0))

In [82]:
vis = {
  'min': 0.0,
  'max': 3.0,
};


map_test = folium.Map(location=[-5,110], zoom_start=6)


map_test.add_ee_layer(dataset, vis, 'test')

display(map_test)

In [36]:
task = ee.batch.Export.image.toDrive(image=dataset,
                                     description='SEA_water_recurrence',
                                     crs='EPSG:4326',
                                     scale=SEAsia_extent.projection().nominalScale(),
                                     fileFormat='GeoTIFF')
task.start()

In [37]:
task.status()

{'state': 'READY',
 'description': 'SEA_water_recurrence',
 'creation_timestamp_ms': 1617148384777,
 'update_timestamp_ms': 1617148384777,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'WJKVFHEBAYCDVYOSPPRBZIJB',
 'name': 'projects/earthengine-legacy/operations/WJKVFHEBAYCDVYOSPPRBZIJB'}